# Imports

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone
import openai
from langchain.document_loaders import DirectoryLoader, Docx2txtLoader, TextLoader
from langchain.vectorstores import Pinecone


/home/ubuntu/.pyenv/versions/langchain-3.10.2/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from langchain.document_loaders import Docx2txtLoader

# PAths and creds

In [5]:
import sys
sys.path.append('/home/ubuntu/workspace/Creds')
from openai_config import OPENAI_API_KEY, MW_ENVIRONMENT, MW_PINECONE_API_KEY
openai.api_key = OPENAI_API_KEY

In [6]:
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002',
                              openai_api_key= OPENAI_API_KEY)

In [7]:
pinecone_mw_index_name = 'minimal-index-dynamic-chatbot'

In [8]:
pinecone.init(
    api_key= MW_PINECONE_API_KEY,  # find at app.pinecone.io
    environment= MW_ENVIRONMENT,  # next to api key in console
)

# Fns, and variables

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["#","##", "###", "\\n\\n","\\n",".", '\n'],
    chunk_size=1500,
    chunk_overlap=100)

In [ ]:
def return_docs_from_pinecone(pinecone_index, query, embeddings =embeddings):


    '''
    will return top 3 relevant chunks, and also top one, for decision making
    '''
    query_vector = embeddings.embed_query(query)
    docs = pinecone_index.query(query_vector,top_k=3,include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in docs['matches']
    ]
    final_str = ''
    for one in contexts:
        final_str += one
        final_str +='\n\n\n\n'

    return final_str, contexts[0]
    

In [29]:
def update_pinecone_texts(pinecone_index, texts_to_add):
    
    index_name = pinecone_index.configuration.server_variables['index_name']
    updated_index = Pinecone.from_texts(texts_to_add,embeddings, index_name = index_name)
    return updated_index
def update_pinecone_docs(pinecone_index, docs_to_add):

    index_name = pinecone_index.configuration.server_variables['index_name']
    updated_index = Pinecone.from_documents(docs_to_add,embeddings, index_name = index_name)
    return updated_index

# Work

## upserting the index

In [34]:
docx_doc = Docx2txtLoader('/home/ubuntu/workspace/mrwhisper-codespace/ChatBot-Work/web_app/knowledgebase/resolve-readme-summary.docx')


In [35]:
documents = docx_doc.load()

In [36]:
docs = text_splitter.split_documents(documents)

In [37]:
pinecone.list_indexes()

['minimal-index-dynamic-chatbot', 'mw-index-1']

In [15]:
len(docs)

74

In [38]:
docsearch = Pinecone.from_documents(docs, embeddings, index_name=pinecone_mw_index_name)


### Full workflow with pinecone, depiction

In [11]:
sys.path.append('/home/ubuntu/workspace/mrwhisper-codespace/ChatBot-Work/trial_work')

In [14]:
from plain_help import *

In [17]:
docsearch = Pinecone.get_pinecone_index(pinecone_mw_index_name)

In [16]:
update_pinecone_docs, update_pinecone_texts

(<function plain_help.update_pinecone_docs(pinecone_index, docs_to_add, index_name)>,
 <function plain_help.update_pinecone_texts(pinecone_index, texts_to_add, index_name)>)

# RoughWork

In [28]:
docsearch.configuration.server_variables['index_name']

'minimal-index-dynamic-chatbot'

In [23]:
docsearch.configuration.__dict__

{'_base_path': 'https://unknown-unknown.svc.unknown.pinecone.io',
 'server_index': 0,
 'server_operation_index': {},
 'server_variables': {'environment': 'us-central1-gcp',
  'index_name': 'minimal-index-dynamic-chatbot',
  'project_name': 'ad44f5b'},
 'server_operation_variables': {},
 'temp_folder_path': None,
 'access_token': None,
 'api_key': {'ApiKeyAuth': 'e9255438-bf1b-44b1-abf0-1c23307f84db'},
 'api_key_prefix': {},
 'refresh_api_key_hook': None,
 'username': None,
 'password': None,
 'discard_unknown_keys': False,
 'disabled_client_side_validations': '',
 '_disabled_client_side_validations': set(),
 '_Configuration__logger_format': '%(asctime)s %(levelname)s %(message)s',
 'logger_formatter': <logging.Formatter at 0x7f5949c258a0>,
 'logger_stream_handler': None,
 '_Configuration__logger_file': None,
 '_Configuration__debug': False,
 'verify_ssl': True,
 'ssl_ca_cert': '/home/ubuntu/.pyenv/versions/langchain-3.10.2/lib/python3.10/site-packages/certifi/cacert.pem',
 'cert_file':

In [33]:
docsearch.delete(delete_all=True)

{}